In [6]:
import pandas as pd
import json
import requests
from typing import Dict, Any

SCHEDULE_URL_FORMAT = 'http://www.jfa.jp/match/acl_2021/group{}/match/schedule.json'
SCHEDULE_CONTAINER_NAME = 'matchScheduleList'
SCHEDULE_LIST_NAME = 'matchSchedule'

REPLACE_KEY_DICT = {
    'match_date': 'matchDateJpn',
    'section_no': 'matchTypeName',
    'start_time': 'matchTimeJpn',
    'stadium': 'venue',
    'home_team': 'homeTeamName',
    'away_team': 'awayTeamName',
    'status': 'matchStatus'
}
SCORE_DATA_KEY_LIST = {
    'home_goal': 'homeScore',
    'away_goal': 'awayScore',
}

def read_match_json(group: str) -> Dict[str, Any]:
    """各グループの試合リスト情報をjfaのJSON形式で返す
    """
    _url = SCHEDULE_URL_FORMAT.format(group)
    print(f'access {_url}...')
    return json.loads(requests.get(_url).text)

def read_match_df(group: str) -> pd.DataFrame:
    """各グループの試合リスト情報を自分たちのDataFrame形式で返す
    """
    match_list = read_teams(group)[SCHEDULE_CONTAINER_NAME][SCHEDULE_LIST_NAME]
    result_list = []
    match_index_dict = {}
    for _match_data in match_list:
        _row = {}
        _row['group'] = group
        for (target_key, org_key) in REPLACE_KEY_DICT.items():
            _row[target_key] = _match_data[org_key]
        for (target_key, org_key) in SCORE_DATA_KEY_LIST.items():
            _row[target_key] = _match_data['score'][org_key]
        section_no = _row['section_no']
        if section_no not in match_index_dict:
            match_index_dict[section_no] = 1
        else:
            match_index_dict[section_no] += 1
        _row['match_index_in_section'] = match_index_dict[section_no]
        result_list.append(_row)

    return pd.DataFrame(result_list)

match_df = pd.DataFrame()
# 惜しい！ グループFが無かった！
for group in ['G', 'H', 'I', 'J']:
    match_df = pd.concat([match_df, read_match_df(group)])

match_df

access http://www.jfa.jp/match/acl_2021/groupG/match/schedule.json...
access http://www.jfa.jp/match/acl_2021/groupH/match/schedule.json...
access http://www.jfa.jp/match/acl_2021/groupI/match/schedule.json...
access http://www.jfa.jp/match/acl_2021/groupJ/match/schedule.json...


,group,match_date,section_no,start_time,stadium,home_team,away_team,status,home_goal,away_goal,match_index_in_section
0,G,2021/06/22,第1節,17:00,ラジャマンガラスタジアム,浦項スティーラーズ,ラチャブリFC,試合終了,2,0,1
1,G,2021/06/22,第1節,21:00,ラジャマンガラスタジアム,ジョホール・ダルル・タクジム,名古屋グランパス,試合終了,0,1,2
2,G,2021/06/25,第2節,21:00,ラジャマンガラスタジアム,ラチャブリFC,ジョホール・ダルル・タクジム,試合終了,0,1,1
3,G,2021/06/25,第2節,17:00,ラジャマンガラスタジアム,名古屋グランパス,浦項スティーラーズ,試合終了,3,0,2
4,G,2021/06/28,第3節,17:00,ラジャマンガラスタジアム,ラチャブリFC,名古屋グランパス,試合終了,0,4,1
5,G,2021/06/28,第3節,21:00,ラジャマンガラスタジアム,浦項スティーラーズ,ジョホール・ダルル・タクジム,試合終了,4,1,2
6,G,2021/07/01,第4節,21:00,ラジャマンガラスタジアム,名古屋グランパス,ラチャブリFC,試合終了,3,0,1
7,G,2021/07/01,第4節,17:00,ラジャマンガラスタジアム,ジョホール・ダルル・タクジム,浦項スティーラーズ,試合終了,0,2,2
8,G,2021/07/04,第5節,17:00,ラジャマンガラスタジアム,ラチャブリFC,浦項スティーラーズ,試合終了,0,0,1
9,G,2021/07/04,第5節,21:00,ラジャマンガラスタジアム,名古屋グランパス,ジョホール・ダルル・タクジム,試合終了,2,1,2
